# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Miguel Ángel Álvarez Cabanes <br>
Url: https://github.com/maalvarezcabanes/algoritmos_optimizacion <br>
Problema:Trabajo práctico - Sesiones de doblaje <br>

## Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

Los datos son:
* Número de actores: 10
* Número de tomas: 30
* Actores/Tomas: https://bit.ly/36D8IuK
* Notación: 1 indica que el actor participa en la toma, 0 en caso contrario


(*) La respuesta es obligatoria

## Pregunta 1
(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>

¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




### Respuesta

Si no existiese ningún tipo de restricción, en mi opinión y desde el punto de vista de minimización de coste solo existiría una posibilidad que consiste en que se grabasen todas las sesiones el mismo día y tampoco creo que tenga una importancia especial el orden en el que se hiciesen. Básicamente tendríamos una combinación de 30 elementos en las que cogeríamos 30 elementos, por lo tanto $$ C^{30}_{30}=\frac{30!}{30!0!}=1 $$

Si consideramos que no hay restricciones pero el orden es significativo, tendríamos variaciones (aunque como cogemos todos en el fondo sería una permutación y por tanto tendríamos un número de soluciones: $$ V^{30}_{30}=\frac{30!}{0!}=2.65e^{+32} $$

Sin embargo, dada la restricción de 6 tomas por día, y teniendo en cuenta que el orden de tomas por día no es relevante, el espacio de soluciones sería: $$ C^{6}_{30} C^{6}_{24} C^{6}_{18} C^{6}_{12} C^{6}_{6} = \frac{30!24!18!12!6!}{6!24!6!18!6!12!6!6!6!0!} = \frac{30!}{5*6!} = 73681349947830849621196800000 $$

## Pregunta 2
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


### Respuesta

El modelo que he elegido como estructura de datos es una lista doble en la que el primer nivel representa los días (1-5) y el segundo nivel los identificadores de sesiones del día.

Creo que es una buena estructura de datos ya que en el fondo las diferentes soluciones del problema parten de la reasignación de las diferentes sesiones a los diferentes días. Probablemente en un modelo de producción y dado que en mi día a día suelo utilizar programación orientada a objetos habría trabajado con una clase Sesion y una clase Dia y habría gestionado los costes dentro de los objetos de la segunda clase (reduciendo algunas operaciones que realizo varias veces), pero creo que en un notebook como este la solución habría quedado más ilegible.

## Pregunta 3
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

### Respuesta

Yo he partido de considerar que la función de coste es: $$ Coste_{total} = sum^{5}_{i=1} Coste\_dia(i) $$
donde $$ Coste\_dia = Número\_actores\_usados\_en\_día $$

El sumatorio es de 1 a 5 porque con las restricciones establecidas no se puede obtener la solución en menos de 30/6 = 5 días y ninguna solución empleando más de 5 días va a ser más óptima que una que use 5 días.

Tal como he planteado el problema, considero que es un problema de minimización (de coste)

## Pregunta 4
Diseña un algoritmo para resolver el problema por fuerza bruta

### Notas y funciones auxiliares
Nota: Voy incluyendo a medida que va discurriendo el notebook funciones que aunque no serían extríctamente necesarias para el apartado sí van a ayudar a la coherencia del problema global.

In [7]:
import numpy as np
import pandas as pd
import random
import os
import logging
import time

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [8]:
def clean_plot(ax, xlabel, ylabel, title, legend = True, rotate = 45):
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    if legend:
        ax.legend(bbox_to_anchor=(1.2, 1))
    if rotate:
        ax.tick_params(axis='x', labelrotation = rotate)

In [9]:
logging.basicConfig(level=logging.INFO)
logging.getLogger().setLevel(logging.INFO) # Configuración del nivel de trazas

show_expanded_solution = True # Activación de la impresión de solución expandida

limite_sesiones_actor = 6 # Limite de sesiones por actor y día

In [10]:
def init_problem(problem_file):
    ''' Esta función se utiliza para inicializar los datos del problema y asegurarnos de que siempre partimos de una imagen fresca cada intento de aproximación '''
    df = pd.read_excel(problem_file, skiprows=1).drop(["Unnamed: 11", "Total"], axis=1)
    df.set_index("Toma", inplace=True)
    df.drop("TOTAL", inplace=True)
    df.dropna(inplace=True)
    total_sesiones_por_actor = np.array(df.sum()).astype(np.int32)
    return (df, total_sesiones_por_actor)

In [13]:
def minimo_coste_hipotetico(df, total_sesiones_por_actor, limite_sesiones_actor):
    ''' Cálculo de un coste mínimo hipotético que se obtendría si se pudiesen asignar los actores a las tomas de forma independiente '''
    coste_minimo_actor = -(total_sesiones_por_actor // -limite_sesiones_actor)
    print(f"Coste mínimo por actor: \n{coste_minimo_actor}")
    coste_minimo = np.sum(coste_minimo_actor)
    print(f"Coste mínimo hipotético: {coste_minimo}")
    return (max(coste_minimo_actor), coste_minimo)

### Lectura de datos de tomas y actores
Los datos originales están en https://docs.google.com/spreadsheets/d/1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0/edit#gid=0, pero los descargo a un fichero "doblaje.xlsx" por comodidad.

A continuación leo los datos con pandas limpiando el DataFrame para solo quedarme con los datos de actores y tomas

In [11]:
(df, total_sesiones_por_actor) = init_problem(os.path.join(".", "doblaje.xlsx"))
display(df)
print(f"Total sesiones por actor: {total_sesiones_por_actor}")

,1,2,3,4,5,6,7,8,9,10
Toma,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
6,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
7,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
8,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Total sesiones por actor: [22 14 13 15 11  8  3  4  2  2]


### Mínimo coste hipotético
Para establecer un mínimo coste hipotético, voy a sumar las sesiones de cada actor y lo voy a dividir entre seis. Hipotéticamente si los actores pudiesen grabar las sesiones de forma independiente, la suma de esos costes sería el mínimo. Es probable que esa solución no sea factible, pero sabemos seguro que ninguna solución va a ser mejor que esa.

In [14]:
dias_hipotetico, coste_hipotetico = minimo_coste_hipotetico(df, total_sesiones_por_actor, limite_sesiones_actor)

Coste mínimo por actor: 
[4 3 3 3 2 2 1 1 1 1]
Coste mínimo hipotético: 21


### Algoritmo de fuerza bruta


## Pregunta 5
Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

## Pregunta 6
(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

## Pregunta 7
(*)Calcula la complejidad del algoritmo

Respuesta

## Pregunta 8
Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

## Pregunta 9
Aplica el algoritmo al juego de datos generado

Respuesta

## Pregunta 10
Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

## Pregunta 11
Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta